# Installations and imports

In [ ]:
# !pip3 install transformers
# !pip3 install torch
# !pip3 install datasets
# !pip3 install sentencepiece
# !pip3 install gdown
# !pip3 install accelerate -U

In [ ]:
import torch
from transformers import (
    LlamaForCausalLM, LlamaConfig, LlamaTokenizer,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from datasets import load_dataset
import sentencepiece as spm
import os
import logging
import json
import sys
import argparse

In [ ]:
def train_tokenizer(input_path, model_prefix):
    spm.SentencePieceTrainer.train(
        input=input_path,
        model_prefix=model_prefix,
        model_type="BPE"
    )

In [ ]:
def move_tokenizer_to_folder(source, destination_folder):
    os.rename(source, os.path.join(destination_folder, "tokenizer.model"))

def create_config_file(folder_path, content):
    with open(os.path.join(folder_path, "config.json"), "w") as config_file:
        json.dump(content, config_file, indent=4)

In [ ]:
config_content = {
    "_name_or_path": "./names_1m",
    "architectures": [
        "LlamaForCausalLM"
    ],
    "bos_token_id": 2,
    "eos_token_id": 3,
    "hidden_act": "silu",
    "hidden_size": 64,
    "initializer_range": 0.02,
    "intermediate_size": 180,
    "max_position_embeddings": 32,
    "model_type": "llama",
    "num_attention_heads": 16,
    "num_hidden_layers": 8,
    "num_key_value_heads": 16,
    "pad_token_id": 1,
    "pretraining_tp": 1,
    "rms_norm_eps": 1e-06,
    "rope_scaling": None,
    "tie_word_embeddings": False,
    "torch_dtype": "float32",
    "transformers_version": "4.28.1",
    "use_cache": False,
    "vocab_size": 97
}


out_folder_path = "bookAndGenre"
os.makedirs(out_folder_path, exist_ok=True)
create_config_file(out_folder_path, config_content)
train_tokenizer('bookData.csv', 'tokenizer')
move_tokenizer_to_folder("tokenizer.model", out_folder_path)

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(out_folder_path)
tokenizer.pad_token = tokenizer.eos_token

# Setting up the model with config
* Function created to return the model with required config

In [ ]:
def create_config_model(path):
    config = LlamaConfig.from_pretrained(path)

    model = LlamaForCausalLM(config)

    if torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model_size = sum(t.numel() for t in model.parameters())

    print(f"GPT Model size: {model_size/1000**2:.1f}M parameters")
    
    return model

# Setting up training the model
* Function created to setup training for the model

In [ ]:
def train_model(model, tokenizer, train_dataset, test_dataset, out_folder_path):
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=out_folder_path,
        overwrite_output_dir=True,
        num_train_epochs=100,
        per_device_train_batch_size=8,
        save_steps=10000,
        logging_steps=10,
        eval_steps=1000,
        logging_dir=f'{out_folder_path}/logs',
        evaluation_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.001)]
    )

    trainer.train()
    model.save_pretrained(out_folder_path)

In [ ]:
def create_tokenized_dataset_splits(path, tokenizer, block_size):
    dataset = load_dataset('text', data_files=path)
    shuffled_dataset = dataset['train'].shuffle(seed=5)
    split_datasets = shuffled_dataset.train_test_split(test_size=0.2)

    def tokenize_dataset(dataset):
        return dataset.map(
            lambda examples: tokenizer(
                examples['text'], truncation=True,
                padding='max_length', max_length=block_size
            ),
            batched=True
        )

    return tokenize_dataset(split_datasets['train']), tokenize_dataset(split_datasets['test'])

In [13]:
model = create_config_model(out_folder_path)
train_dataset, test_dataset = create_tokenized_dataset_splits('bookData.csv', tokenizer, block_size=32)
train_model(model, tokenizer, train_dataset, test_dataset, out_folder_path)


                                                              

                                                       
  0%|          | 1000/527553800 [34:35<12866:41:25, 11.39it/s]



{'loss': 1.0381, 'grad_norm': 1.3139171600341797, 'learning_rate': 4.999998862675238e-05, 'epoch': 0.0}






                                                              

                                                       
  0%|          | 1000/527553800 [34:36<12866:41:25, 11.39it/s]



{'loss': 1.0241, 'grad_norm': 0.822917640209198, 'learning_rate': 4.9999987678981746e-05, 'epoch': 0.0}






                                                              

                                                       
  0%|          | 1000/527553800 [34:37<12866:41:25, 11.39it/s]



...

{'loss': 0.9325, 'grad_norm': 0.8698910474777222, 'learning_rate': 4.999990711847778e-05, 'epoch': 0.0}






                                                              

                                                       
  0%|          | 1000/527553800 [35:56<12866:41:25, 11.39it/s]



{'loss': 0.8398, 'grad_norm': 1.882041335105896, 'learning_rate': 4.9999906170707136e-05, 'epoch': 0.0}






                                                              

                                                       
  0%|          | 1000/527553800 [35:57<12866:41:25, 11.39it/s]


{'loss': 0.9089, 'grad_norm': 0.6111797094345093, 'learning_rate': 4.9999905222936506e-05, 'epoch': 0.0}


In [ ]:
def generateStory(model, tokenizer, prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    attention_mask = torch.ones_like(input_ids).to(model.device)

    with torch.no_grad():
        output = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=15,
                early_stopping=True,
                temperature=0.6,
                top_p=0.8,
                top_k=50,
                do_sample=True,
                output_scores=True,
                pad_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.4,
                eos_token_id=tokenizer.eos_token_id
            )
        output_str = tokenizer.decode(output[0], skip_special_tokens=True).split(".")[0]
        print(output_str)

# Output 

In [ ]:
model.eval()
generateStory(model, tokenizer, "adventure")